# Attention
Class = 4.

For now, process all images. Consider using only the WSI training set.

Reduce the nuclei to those in patches predicted correctly by CNN.

Integrate (1) Nucleus features as recored by CellProfiler Proces100
and (2) attention level recorded by the CNN.
Use Random Forest to say what CP features are predictive of attention.
Here, we do the analysis per class, just because that is easiest.

In [1]:
import datetime
import numpy as np
import pandas as pd
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
THISCLASS=4
CLASSPATH='/home/jrm/Adjeroh/Naved/CP_80K/Output4/'
NUCFILENAME='Process100_Nucleus.csv'
PATCHFILENAME='Process100_Image.csv'
ATTFILENAME='attention_avg_temp_per_nucleus.csv'
CNNFILENAME='/home/jrm/Adjeroh/Naved/CP_80K/CNN/scores.csv'

## Load CNN scores
Retain filenames of patches predicted correctly by the CNN

In [3]:
scores=pd.read_csv(CNNFILENAME)  # same file for all classes
scores

,patchname,ground_truth,predicted_label,softmax_0,softmax_1,softmax_2,softmax_3,softmax_4,softmax_5
0,TCGA-02-0004-01Z-00-DX1_4800_3600,0,0,9.999977e-01,2.466497e-08,1.495720e-07,7.797728e-08,6.044772e-08,1.899641e-06
1,TCGA-02-0004-01Z-00-DX1_4800_4200,0,0,9.999999e-01,4.827287e-13,4.781109e-11,1.163105e-14,1.071110e-11,8.704473e-08
2,TCGA-02-0004-01Z-00-DX1_4500_4200,0,0,1.000000e+00,1.984801e-10,8.819246e-09,1.731268e-09,6.153791e-11,1.787250e-09
3,TCGA-02-0004-01Z-00-DX1_5400_3300,0,0,9.999882e-01,3.108451e-07,1.531888e-06,3.748598e-06,1.214340e-06,5.000012e-06
4,TCGA-02-0004-01Z-00-DX1_5400_3900,0,0,9.999981e-01,1.063941e-14,7.136163e-12,6.937229e-16,2.445667e-11,1.867779e-06
...,...,...,...,...,...,...,...,...,...
82965,TCGA-S9-A7J1-01Z-00-DX1_72600_26400,1,2,3.380104e-07,2.480321e-02,8.634496e-01,6.795154e-07,2.734306e-03,1.090119e-01
82966,TCGA-S9-A7J1-01Z-00-DX1_69600_58500,1,4,6.624015e-08,1.844222e-03,1.204252e-01,1.170510e-05,5.556598e-01,3.220590e-01
82967,TCGA-S9-A7J1-01Z-00-DX1_70800_48300,1,4,1.643103e-07,1.568889e-03,3.065287e-01,2.030722e-05,5.148395e-01,1.770425e-01
82968,TCGA-S9-A7J1-01Z-00-DX1_74400_60600,1,4,3.034546e-06,1.155498e-04,1.003913e-01,3.831314e-05,8.207427e-01,7.870915e-02


In [4]:
print('before',scores.shape)
scores.drop(columns=['softmax_0','softmax_1','softmax_2'],inplace=True)
scores.drop(columns=['softmax_3','softmax_4','softmax_5'],inplace=True)
scores.drop(scores[scores.ground_truth!=THISCLASS].index,inplace=True)
scores.drop(scores[scores.predicted_label!=THISCLASS].index,inplace=True)
print('after',scores.shape)
scores

before (82970, 9)
after (785, 3)


,patchname,ground_truth,predicted_label
65282,TCGA-DB-A4XF-01Z-00-DX1_1200_23700,4,4
65284,TCGA-DB-A4XF-01Z-00-DX1_5400_22200,4,4
65290,TCGA-DB-A4XF-01Z-00-DX1_9300_16200,4,4
65321,TCGA-DB-A4XF-01Z-00-DX1_15900_28200,4,4
65416,TCGA-DB-A4XF-01Z-00-DX1_46200_24900,4,4
...,...,...,...
68455,TCGA-S9-A6WL-01Z-00-DX1_48000_28500,4,4
68457,TCGA-S9-A6WL-01Z-00-DX1_51900_21900,4,4
68458,TCGA-S9-A6WL-01Z-00-DX1_49200_28500,4,4
68460,TCGA-S9-A6WL-01Z-00-DX1_43200_18000,4,4


## Load images
Retain mapping between patch filename and cell profiler image number.

In [12]:
patch_all = pd.read_csv(CLASSPATH+PATCHFILENAME)

In [14]:
patch_map1 = patch_all[ ['ImageNumber','FileName_Tumor'] ]
patch_map2 = patch_map1.rename(
    columns={'ImageNumber':'PatchNumber','FileName_Tumor':'patchname'})
patch_map2['patchname'] = patch_map2['patchname'].str[:-4]
patch_map2

,PatchNumber,patchname
0,1,TCGA-DB-A4XF-01Z-00-DX1_10200_20700
1,2,TCGA-DB-A4XF-01Z-00-DX1_10200_28500
2,3,TCGA-DB-A4XF-01Z-00-DX1_10200_37500
3,4,TCGA-DB-A4XF-01Z-00-DX1_10500_27900
4,5,TCGA-DB-A4XF-01Z-00-DX1_10500_39300
...,...,...
3192,3193,TCGA-S9-A6WL-01Z-00-DX1_9900_35400
3193,3194,TCGA-S9-A6WL-01Z-00-DX1_9900_35700
3194,3195,TCGA-S9-A6WL-01Z-00-DX1_9900_36000
3195,3196,TCGA-S9-A6WL-01Z-00-DX1_9900_36300


## Load labels
The labels are the temperatures of the CNN attention heatmap,
specifically an average across the bounding box of each nucleus.

In [4]:
att_df=pd.read_csv(ATTFILEPATH+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
att_df.rename(columns=cols,inplace=True)
att_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
att_df.reset_index(drop=True,inplace=True)

In [5]:
# Later, try the RandomForestRegressor. For now use bins and classification.
bins = [0, 0.3, 0.45, 0.6, 0.75, 1.0]
labels = ['to30%', 'to45%', 'to60%', 'to75%', 'to100%']
att_df['bin'] = pd.cut(att_df['AvgTemp'], bins=bins, labels=labels)
att_df

,PatchNumber,ObjectNumber,AvgTemp,bin
0,1,1,0.024352,to30%
1,1,2,0.167493,to30%
2,1,3,0.301966,to45%
3,1,4,0.568698,to60%
4,1,5,0.341957,to45%
...,...,...,...,...
46520,3196,6,0.542161,to60%
46521,3196,7,0.419783,to45%
46522,3196,8,0.519468,to60%
46523,3196,9,0.564878,to60%


In [6]:
ytrain = att_df['bin'] 
ytrain.value_counts()

to30%     10295
to100%    10230
to60%      9689
to45%      8818
to75%      7493
Name: bin, dtype: int64

## Load features
These are features of nuclei as determined by our "Process100" pipeline for CellProfiler.
These are the nuclei used to set the bounding boxes for the temperature lables.
Other than that, the features and temperatures were each derived independently.

In [7]:
nuc_df=pd.read_csv(NUCFILEPATH+NUCFILENAME)
cols={'ImageNumber':'PatchNumber'}
nuc_df.rename(columns=cols,inplace=True)
nuc_df.sort_values(by=['PatchNumber','ObjectNumber'],axis=0,inplace=True)
nuc_df.reset_index(drop=True,inplace=True)

In [8]:
Xtrain = nuc_df.drop(columns=['PatchNumber','ObjectNumber'])
Xtrain

,AreaShape_Area,AreaShape_BoundingBoxArea,AreaShape_BoundingBoxMaximum_X,AreaShape_BoundingBoxMaximum_Y,AreaShape_BoundingBoxMinimum_X,AreaShape_BoundingBoxMinimum_Y,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,...,Texture_Variance_Hematoxylin_4_02_256,Texture_Variance_Hematoxylin_4_03_256,Texture_Variance_Hematoxylin_5_00_256,Texture_Variance_Hematoxylin_5_01_256,Texture_Variance_Hematoxylin_5_02_256,Texture_Variance_Hematoxylin_5_03_256,Texture_Variance_Hematoxylin_7_00_256,Texture_Variance_Hematoxylin_7_01_256,Texture_Variance_Hematoxylin_7_02_256,Texture_Variance_Hematoxylin_7_03_256
0,196,294,80,26,59,12,68.193878,18.607143,196.0,-5.329071e-15,...,602.743285,642.679273,642.319285,642.400310,640.764599,699.330009,644.967400,687.076735,834.618344,850.561523
1,469,609,126,37,97,16,111.159915,25.989339,469.0,-9.166001e-13,...,539.175397,565.853063,548.903754,562.875134,556.688253,597.419939,567.698283,508.662662,579.599297,624.531535
2,223,304,193,41,174,25,182.551570,32.852018,223.0,4.796163e-14,...,888.080613,808.414113,845.188775,847.659739,899.641109,733.203961,877.066057,587.996061,814.744709,819.948881
3,508,798,21,61,0,23,7.982283,43.293307,508.0,2.131628e-14,...,1241.667208,1159.372486,1238.719919,1169.194520,1257.374985,1114.919318,1175.562484,1175.745806,1287.354528,1171.277847
4,521,768,143,74,111,50,126.280230,60.255278,521.0,-1.652012e-13,...,951.986476,964.896780,907.957406,909.009379,956.391792,966.036051,933.923255,943.625640,974.996622,928.684668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46520,491,684,234,216,216,178,224.256619,197.356415,491.0,7.105427e-14,...,1422.754541,1428.622175,1361.131323,1573.083123,1431.157207,1464.698052,1423.252150,1665.083694,1452.185763,1378.132307
46521,298,364,149,208,136,180,142.043624,193.359060,298.0,-2.842171e-14,...,1695.628540,1787.881929,1789.764515,1910.202155,1681.800641,1907.651012,2040.167169,2072.963190,1667.108287,2022.937578
46522,235,345,210,219,195,196,200.523404,206.736170,235.0,-6.039613e-14,...,1399.381486,1530.439781,1401.851562,1347.494422,1361.041939,1664.847698,1485.990710,1525.224615,1400.377914,1819.843827
46523,269,414,214,233,196,210,205.635688,222.234201,269.0,-2.113865e-13,...,1255.160768,1348.841787,1194.540566,1004.038428,1251.985997,1453.395007,1280.273024,901.265840,1300.529218,1757.844843


In [9]:
att_df=None
nuc_df=None
print('Count Nan before:',Xtrain.isna().sum().sum())
Xtrain.fillna(0,inplace=True)
print('Count Nan after:',Xtrain.isna().sum().sum())

Count Nan before: 139801
Count Nan after: 0


## Measure agreement (with a random forest classifier)
Can the RF predict the temperature of each nucleus given only the CellProfiler features?

In [10]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-09 16:34:13.694652
Shuffle...


In [11]:
print(datetime.datetime.now())
print('Cross validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('Accuracy mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-09 16:34:13.791301
Cross validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

## Measure feature importance

In [ ]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf1 = None
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
rf2.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf2.important_features()
pd.set_option('display.max_rows', None)
top.loc[:200]